# BERT baseline for MetaHate

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm

## Reading the data

In [ ]:
data = pd.read_csv('/data/metahate.csv', sep='\t', names=['label', 'text'])

texts = data['text'].tolist()
labels = data['label'].tolist()

## Splitting the data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    texts,              # Input features (text data)
    labels,             # Target labels corresponding to the input features
    test_size=0.2,      # The proportion of the dataset to include in the test split (20% in this case)
    random_state=42     # Seed for reproducibility (set to 42 in this case)
)

## Loading BERT model

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

model = BertForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2 # Configuring the model for binary classification (2 labels)
)

## Tokenizing the training and testing data

In [ ]:
X_train_tokens = tokenizer(
    X_train.tolist(), 
    padding=True, # Padding the sequences to ensure uniform length
    truncation=True, # Truncating sequences to the maximum length supported by the model
    return_tensors='pt' # Returning PyTorch tensors as the output
)

X_test_tokens = tokenizer(
    X_test.tolist(), 
    padding=True, # Padding the sequences to ensure uniform length
    truncation=True, # Truncating sequences to the maximum length supported by the model
    return_tensors='pt' # Returning PyTorch tensors as the output
)

## Converting the training and testing labels to PyTorch tensors

In [ ]:
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

## Defining class weights for imbalanced classes

In [ ]:
class_weights = torch.tensor([0.63, 2.42]) # Hate texts have a higher weight as are only the 20.64% of the data.
loss_function = nn.CrossEntropyLoss(weight=class_weights)

## Creating PyTorch TensorDatasets and DataLoaders for training and testing sets

In [ ]:
train_dataset = TensorDataset(X_train_tokens['input_ids'], X_train_tokens['attention_mask'], y_train_tensor)
test_dataset = TensorDataset(X_test_tokens['input_ids'], X_test_tokens['attention_mask'], y_test_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Training

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5) # Setting up AdamW optimizer for model parameters with a learning rate of 5e-5

# Checking for the availability of a GPU and moving the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 3

# Training through 3 epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    # Iterating over batches in the training data
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_function(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f'Training Loss: {average_loss}')


## Evaluate

In [ ]:
model.eval()
all_predictions = []
with torch.no_grad(): # Disabling gradient computation during evaluation
    
    # Iterating over batches in the testing dataloader
    for batch in tqdm(test_dataloader, desc='Evaluating'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()
        all_predictions.extend(predictions)

## Evaluate the model

In [ ]:
accuracy = accuracy_score(y_test, all_predictions)
report = classification_report(y_test, all_predictions)
weighted_f1 = f1_score(y_test, all_predictions, average='weighted')
micro_f1 = f1_score(y_test, all_predictions, average='micro')
macro_f1 = f1_score(y_test, all_predictions, average='macro')

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
print(f"Weighted F1 Score: {weighted_f1}")
print(f"Micro F1 Score: {micro_f1}")
print(f"Macro F1 Score: {macro_f1}")